
---

# **Project Title**  
**Hopfield-Network-Based Multi-Robot Coordination for Collective Object Transport in CoppeliaSim**

---

## **1. Goal and Scope**

### **1.1 Goal**  
- To design, implement, and analyze a **Hopfield network** that controls a swarm of \(N\) robots (all attached beneath a shared object) in **CoppeliaSim**.  
- The Hopfield network encodes high-level, discrete **movement modes** (forward, backward, turning, etc.) to ensure the swarm collectively transports the object to target orientations or positions under **delays** or **disturbances**.

### **1.2 Scope**  
- **In Scope**  
  - Modeling \(N\) mobile robots (differential- or omni-wheel) in CoppeliaSim, each rigidly attached beneath one large platform or object.  
  - A centralized or unified **Hopfield network** that outputs which “motion option” the swarm should follow (forward, backward, turn left, turn right, etc.).  
  - Analysis of performance under intrinsic delays (actuator or sensor lags) and extrinsic disturbances (uneven friction or small random forces).  
  - Data collection (graphs, plots) to evaluate convergence time, positional/orientation accuracy, and robustness to delay.

- **Out of Scope**  
  - Hardware implementation.  
  - Dynamic reconfiguration (the adjacency/configuration of the robots under the platform is **fixed** from the start).

---

## **2. Introduction and Motivation**

When multiple robots are **physically attached** to a single object, collision avoidance is unnecessary. However, **coordinating motion** becomes critical—one robot attempting to move forward while another moves in the opposite direction can cause mechanical stress or ineffective transport. A **Hopfield network** offers a **recurrent neural approach** to converge on a common “movement option” that satisfies the group’s task.

1. **Hopfield Network Basics**  
   - A **Hopfield network** typically encodes an energy function. It converges to “stable states” that minimize that energy, naturally enforcing constraints (e.g., not choosing conflicting movement modes).
2. **Decentralized vs. Centralized**  
   - We can implement the Hopfield network as a single “global brain” that determines the swarm’s movement mode. Each robot then sets wheel commands accordingly.  
   - Alternatively, each robot could have a local Hopfield module that collectively converges to the same state. However, here we focus on **one** integrated Hopfield network controlling the group’s motion modes, since all robots are rigidly connected anyway.
3. **Practical Relevance**  
   - Factory or warehouse applications often involve large loads on top of multiple motorized modules. A Hopfield-based approach can unify these modules without complicated hand-tuned logic, offering robust, delay-tolerant control.

---

## **3. Tasks and Subtasks**

### **3.1 System Modeling and Setup**
1. **Robot-Object Configuration**  
   - In CoppeliaSim, place \(N\) robots (likely differential-drive) beneath a shared platform.  
   - Use **rigid attachments** so that their relative positions are fixed, ensuring no collisions.  
2. **Environment**  
   - Define friction parameters, sensor placeholders (if needed), and possible environment boundaries for reference.

### **3.2 Hopfield Network Design**
1. **Discrete Movement Modes**  
   - Define a set of “options”: \(\{\text{Forward}, \text{Backward}, \text{Turn Left}, \text{Turn Right}, \text{Pivot}, \ldots\}\).  
   - These options are **mutually exclusive** (in most cases).  
2. **Energy Function**  
   - Encode a **desired direction** (the current goal) as a bias that “pulls” the network toward the corresponding movement mode.  
   - Enforce **mutual exclusivity** with negative weights between incompatible modes (e.g., Forward vs. Backward).  
   - Optionally, allow partial concurrency (e.g., “Forward + slight Turn Left”) by adjusting weights to be less negative, if desired.
3. **Handling Delays & Disturbances**  
   - Add simulated lags to motor response or sensor feedback.  
   - Ensure the network can still converge robustly to one stable mode even if updates arrive with latency or contain noise.

### **3.3 Behavior Implementation in CoppeliaSim**
1. **Control Loop**  
   - At each decision step, run the Hopfield network until it stabilizes on a single mode (or combination).  
   - Translate the chosen mode into **specific wheel commands** for each of the \(N\) robots. For example, “Forward” = leftWheelSpeed = rightWheelSpeed = \(v\).  
2. **Goal Sequence**  
   - Script a sequence of movements (forward for 2 meters, then turn 90 degrees left, etc.) to test how well the Hopfield approach transitions between modes.  

### **3.4 Data Collection and Analysis**
1. **Performance Metrics**  
   - **Orientation & Position Error**: How close the object is to the intended final pose.  
   - **Convergence Time**: Number of iterations (or simulation steps) for the Hopfield network to lock onto a single movement mode.  
   - **Sensitivity to Delay**: Vary the magnitude of actuator/sensor delays and log success/failure rates or time overhead.  
2. **Graphs and Plots**  
   - **Hopfield Activation Levels** over time: shows which movement mode is dominating at each update step.  
   - **Object Trajectories** in the simulator (2D overhead or 3D views).  
   - **Bar/Line Charts** comparing performance under different disturbance levels.

---

## **4. Theory and Mathematical Equations**

### **4.1 Hopfield Network Overview**
A Hopfield network has a set of units (neurons) \(\{x_1, x_2, \ldots, x_M\}\). In this project, each unit \(x_i\) corresponds to one **movement mode**. We define:

\[
E(\mathbf{x}) 
= -\frac{1}{2}\sum_{i=1}^M \sum_{j=1}^M w_{ij} \, x_i \, x_j 
\;+\; \sum_{i=1}^M \theta_i \, x_i,
\]

where

- \(w_{ij}\) are the symmetric weights dictating compatibility between modes \(i\) and \(j\).  
- \(\theta_i\) is a bias (e.g., from the current goal: if we want to move forward, \(\theta_{\text{forward}}\) is high).  
- \(\mathbf{x}\) can be **binary** (e.g., \(\pm1\)) or continuous (using a sigmoid \(\sigma\)).

Minimizing \(E(\mathbf{x})\) leads the network to pick a stable configuration of the \(\{x_i\}\). Because we have **mutually exclusive** modes, we typically want exactly one unit to be “on” or strongly positive, and the rest “off.”

### **4.2 Update Rule**
A common Hopfield update is asynchronous:

\[
x_i(t+1) = \sigma\!\biggl(\sum_{j=1}^M w_{ij} \, x_j(t) + \theta_i\biggr),
\]

where

- \(\sigma\) is a sign function (for binary Hopfield) or logistic/tanh (for continuous).  
- The network iterates until stability, which means \(\forall i,\; x_i(t+1) \approx x_i(t)\).

### **4.3 Translating Mode Selection to Robot Speeds**
Once the network converges on a dominant mode \(k\),

\[
x_k \approx 1 \quad \text{and} \quad x_{j \neq k} \approx 0,
\]

we apply a **pre-mapped** set of wheel velocities for that mode:

\[
(\omega_{r1}, \omega_{r2}) = \Psi(O_k, r),
\]

- \(\omega_{r1}, \omega_{r2}\) are the left/right wheel speeds for robot \(r\).  
- \(\Psi\) is a lookup table: e.g., \(\Psi(\text{Forward}, r)\) = \((v, v)\), \(\Psi(\text{Turn Left}, r)\) = \((v, -v)\), etc., possibly with small offsets if the robots are arranged in a particular formation.

---

## **5. Expected Outcomes**

1. **Seamless Collective Movement**  
   - The object is moved forward, backward, rotated, etc., with all attached robots acting coherently.  
   - No physical collisions occur, as they share a single rigid platform.

2. **Robustness Against Delays**  
   - Plots showing that, within reasonable actuator/sensor lag, the Hopfield network still converges to correct movement modes.  
   - Possibly identify upper-bound delay thresholds beyond which performance degrades or movement becomes erratic.

3. **Clear Visual Data**  
   - **Time-series graphs** of Hopfield neuron activations, illustrating how the network picks each movement mode.  
   - **Trajectory plots** showing the object’s path during each movement phase.  
   - **Performance charts** comparing time-to-goal or final orientation error under varying disturbance levels.

---

## **6. Timeline (Six Months Total, Last Month = Writing)**

| **Task / Month**                     | **1** | **2** | **3** | **4** | **5** | **6 (Writing)** |
|--------------------------------------|:----:|:----:|:----:|:----:|:----:|:----------------:|
| **1. Literature & Env. Setup**       |  X   |  X   |      |      |      |        -         |
| **2. Robot & Object Configuration**  |  X   |  X   |  X   |      |      |        -         |
| **3. Hopfield Network Implementation**|     |  X   |  X   |  X   |      |        -         |
| **4. Testing Delays & Disturbances** |     |     |  X   |  X   |  X   |        -         |
| **5. Data Collection & Analysis**    |     |     |     |  X   |  X   |        -         |
| **6. Final Documentation**           |     |     |     |     |     |        X          |

1. **Months 1–2**:  
   - **Literature Review**: Gather references on Hopfield networks in robotics.  
   - **CoppeliaSim Setup**: Define the platform and attach \(N\) robots in a rigid arrangement.  
2. **Months 3–4**:  
   - **Hopfield Controller**: Code the network logic (Python or Lua), define movement modes, and test basic scenarios.  
   - **Delays**: Gradually introduce latencies, sensor noise, or friction variations.  
3. **Month 5**:  
   - **Extensive Testing**: Run multiple trials, gather data on convergence time, final pose error, and network stability.  
   - **Visualization**: Plot activation states, trajectories, success rates.  
4. **Month 6**:  
   - **Final Documentation**: Write the thesis/report, compile figures, and prepare slides or posters if required.



### Code Description: Hopfield-Network-Controlled Robot Swarm Simulation

This Python code implements a swarm of differential-drive robots controlled by a Hopfield neural network to achieve coordinated movement. The simulation includes physics-based robot motion, decision-making via energy minimization, and real-time visualization of both swarm behavior and network dynamics.

---


### Core Components

#### 1. DifferentialRobot Class
- Purpose: Models individual robot kinematics and state.
- Key Features:
  - Tracks position (x, y), orientation (theta), and wheel speeds.
  - Implements differential drive physics:
    ```python
    linear_vel = (right_wheel + left_wheel) / 2
    angular_vel = (right_wheel - left_wheel) / wheel_base
    ```
  - Updates pose using Euler integration.

#### 2. HopfieldSwarmController Class
- Purpose: Decision-making engine for swarm coordination.
- Key Features:
  - Hopfield Network:
    - 4 mutually exclusive movement modes: forward, left, right, backward.
    - Weight matrix enforces conflict resolution (e.g., forward inhibits backward).
    - Biases steer the network toward target directions.
  - Convergence:
    - Uses sigmoid activation for continuous state transitions.
    - Stops iterating when network stabilizes (activation changes < 1%).

#### 3. Swarm Class
- Purpose: Manages robot collective behavior.
- Key Features:
  - Initializes robots in a vertical line formation.
  - Translates Hopfield output to wheel speeds:
    ```python
    'forward' → (1.0, 1.0)
    'left' → (-0.5, 0.5)
    ```
  - Updates all robots simultaneously using a fixed timestep (dt = 0.1s).

#### 4. SwarmVisualization Class
- Purpose: Real-time animation of swarm and network states.
- Key Features:
  - Dual-Panel Display:
    - Left: Robot trajectories with:
      - Orientation arrows (color-coded by movement mode).
      - Fading trails (20-step history).
      - Dark theme for contrast.
    - Right: Hopfield network activations:
      - Bar chart showing mode strengths.
      - Yellow/cyan coloring for positive/negative activations.
  - Overlays:
    - Current simulation time.
    - Active movement mode.
    - Network convergence speed.

---

### Workflow
1. Initialization:
   - Create N robots in a line formation.
   - Configure Hopfield weights/biases for mutual exclusivity.

2. Simulation Loop:
   - Every 2.5 seconds: Update target direction (cyclic sequence).
   - Hopfield Convergence:
     - Network iterates until stable state (≤20 steps).
     - Winning mode determines swarm movement.
   - Physics Update: All robots move based on selected mode.
   - Visualization: Redraw arrows, trails, and network state.

3. Visualization:
   - Uses matplotlib.animation.FuncAnimation for smooth updates.
   - Optimized with blit=True and cache_frame_data=False.

---

### Key Algorithms
1. Differential Drive Kinematics:
   - Converts wheel speeds to linear/angular velocity.
   - Updates robot pose using:
     ```python
     x += linear_vel * cos(theta) * dt
     theta += angular_vel * dt
     ```

2. Hopfield Network Dynamics:
   - Energy minimization via recurrent updates:
     ```python
     activation = sigmoid(weights × current_state + biases)
     ```
   - Stable states represent conflict-free movement choices.

---

### Customization Options
1. Swarm Parameters:
   - num_robots: Number of robots (default: 7).
   - target_sequence: Movement goal sequence.
2. Network Parameters:
   - Weights (self.weights[i,j] = -1.5): Conflict strength.
   - Biases (self.biases[0] = 2.0): Target influence.
3. Visualization:
   - trail_length: History length for paths.
   - mode_colors: Color scheme for movement modes.

---

### Execution
1. Dependencies: numpy, matplotlib.
2. Run:
   ```bash
   python swarm_hopfield.py
   ```
3. Output: Interactive window showing:
   - Swarm performing forward → right → forward → left → ... sequence.
   - Real-time Hopfield state transitions.
   - Performance metrics (convergence speed, active mode).

---
